In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow import keras

2.2.0


In [2]:
# It can be used to reconstruct the model identically.
_model = keras.models.load_model("models/lstm_5s_batch22.h5")

In [3]:
#### The Model is an already trained ##########
converter = tf.lite.TFLiteConverter.from_keras_model(_model)
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
converter.experimental_new_converter =True
tflite_model = converter.convert()

In [4]:
open("converted_model.tflite", "wb").write(tflite_model)
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

In [5]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)


[ 1 79 20]


In [6]:
output_details

[{'name': 'Identity',
  'index': 158,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([1, 4], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
input_details

[{'name': 'input',
  'index': 0,
  'shape': array([ 1, 79, 20], dtype=int32),
  'shape_signature': array([ 1, 79, 20], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

## Test the converted model on a sample

In [8]:
X_val = np.load('X_val.npy')
y_val = np.load('y_val.npy')

In [9]:
X_val[1370]

array([[-350.9057   ,   61.24164  ,   37.383137 , ...,  -12.170894 ,
          -3.8168406,   -2.7301626],
       [-360.7419   ,  108.48808  ,   27.992096 , ...,    9.391933 ,
          -1.8722537,  -13.7329445],
       [-351.38797  ,  105.79579  ,   31.297764 , ...,    1.390541 ,
           8.18749  ,    4.534544 ],
       ...,
       [   0.       ,    0.       ,    0.       , ...,    0.       ,
           0.       ,    0.       ],
       [   0.       ,    0.       ,    0.       , ...,    0.       ,
           0.       ,    0.       ],
       [   0.       ,    0.       ,    0.       , ...,    0.       ,
           0.       ,    0.       ]], dtype=float32)

In [11]:
inp = X_val[1370]

In [12]:
inp = np.expand_dims(inp, axis=0)

In [13]:
predicted = _model.predict(inp)

In [14]:
predicted

array([[9.8501694e-01, 6.2197767e-04, 1.4227290e-02, 1.3383293e-04]],
      dtype=float32)

In [15]:
input_data = inp
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[9.8501694e-01 6.2197709e-04 1.4227277e-02 1.3383280e-04]]


In [16]:
np.argmax(output_data[0], axis=0)

0